In [67]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import korean

### 데이터 분석 목표

In [ ]:
# (내가 부동산업자라면) 손님 맞춤형 부동산 추천 서비스
# 특정 구/동 기준으로 괜찮은 집 리스트 제공

# 손님 조건 예시 : 10억 이하, 인프라 조건(병원 접근성, 대형마트 근접, 지하철 접근성)
# 특정구를 노원구로 선택했을 경우
# 서울시 부동산 실거래가 정보(2022~2025).csv 공공데이터 이용

In [69]:
# 공공데이터에서 가져온 노원구에 있는 대형마트 수

nowon_bigmart = pd.read_csv('./data/대형마트_위치.csv', encoding="cp949")
nowon_bigmart_df = nowon_bigmart[nowon_bigmart['자치구'].str.contains('노원구', na=False)]
nowon_bigmart_df

,Unnamed: 0,자치구,법정동,마트개수
2,2,노원구,공릉동,0
32,32,노원구,상계동,0
45,45,노원구,월계동,0
49,49,노원구,중계동,2
52,52,노원구,하계동,0


In [70]:
# 공공데이터에서 가져온 노원구에 있는 지하철 수

nowon_metro = pd.read_csv('./data/지하철_위치.csv', encoding="cp949")
nowon_metro_df = nowon_metro[nowon_metro['자치구'].str.contains('노원구', na=False)]
nowon_metro_df

,Unnamed: 0,자치구,법정동,역개수
2,2,노원구,공릉동,4
10,10,노원구,상계동,6
18,18,노원구,월계동,1
20,20,노원구,중계동,1
22,22,노원구,하계동,1


In [71]:
# 공공데이터에서 가져온 노원구에 있는 병원 수

nowon_hospital = pd.read_csv('./data/최종_노원구_강동구_성북구_병원데이터.csv')
nowon_hospital_df = nowon_hospital[nowon_hospital['자치구'].str.contains('노원구', na=False)]
nowon_hospital_df

,Unnamed: 0,자치구,법정동,병원수
0,1,노원구,상계동,13
1,2,노원구,공릉동,6
2,3,노원구,하계동,1
3,4,노원구,중계동,1
4,5,노원구,월계동,1


In [72]:
# 노원구에서 대형마트가 있는 중계동의 매물 데이터 가져오기
# 서울시 부동산 실거래가 정보(2022~2025).csv 공공데이터

nowon_propertyType = pd.read_csv('./data/서울시 부동산 실거래가 정보(2022~2025).csv', encoding="cp949")
nowon_propertyType_df = nowon_propertyType[nowon_propertyType['자치구명'].str.contains('노원구', na=False)]
nowon_propertyType_df = nowon_propertyType_df[['자치구명', '법정동명', '건물명', '물건금액(만원)', '건물면적(㎡)', '건축년도', '건물용도']]
nowon_propertyType_df

C:\Users\User\AppData\Local\Temp\ipykernel_10008\1240010626.py:4: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  nowon_propertyType = pd.read_csv('./data/서울시 부동산 실거래가 정보(2022~2025).csv', encoding="cp949")


,자치구명,법정동명,건물명,물건금액(만원),건물면적(㎡),건축년도,건물용도
85,노원구,공릉동,스위트휴,26300,29.53,2024.00,연립다세대
92,노원구,공릉동,스위트휴,44600,58.75,2024.00,연립다세대
93,노원구,공릉동,스위트휴,44650,57.81,2024.00,연립다세대
94,노원구,공릉동,스위트휴,40450,54.14,2024.00,연립다세대
95,노원구,공릉동,스위트휴,40450,54.14,2024.00,연립다세대
...,...,...,...,...,...,...,...
235529,노원구,공릉동,NaN,85000,197.28,1991.00,단독다가구
235564,노원구,상계동,호연빌라,50000,58.98,1995.00,연립다세대
235570,노원구,중계동,성원1,88000,59.75,1996.00,아파트
235580,노원구,상계동,대우팰리스,50000,28.10,2017.00,연립다세대


In [73]:
# --- 보기 쉬운 금액 컬럼 추가 ---


# 1️⃣ 보기용 문자열 변환 함수
# - '물건금액(만원)' 숫자를 한국식 금액 문자열(억/만 원)로 변환
# - NaN 값이나 변환 오류가 발생해도 안전하게 처리
def format_korean_money(price_million):
    try:
        # NaN이면 None 반환
        if pd.isna(price_million):
            return None

        # float로 변환 (예: 95000 -> 95000.0)
        price_million = float(price_million)

        # 원 단위로 계산 (1만원 = 10,000원)
        total_won = int(price_million * 10000)

        # 억 단위 계산
        eok = total_won // 100_000_000

        # 만 단위 계산 (억 제외 나머지)
        man = (total_won % 100_000_000) // 10_000

        # 억과 만 모두 있을 때
        if eok > 0 and man > 0:
            return f"{eok}억 {man:,}만 원"
        # 억만 있을 때
        elif eok > 0:
            return f"{eok}억 원"
        # 억 없고 만만 있을 때
        else:
            return f"{man:,}만 원"
    except:
        # 변환 중 오류 발생 시 None 반환
        return None

# 2️⃣ '건축년도' 컬럼을 정수형(Int64)으로 변환
# - Int64는 결측값(NaN)도 허용
# - 원래 데이터가 숫자 또는 NaN이어도 안전하게 변환 가능
nowon_propertyType_df['건축년도'] = nowon_propertyType_df['건축년도'].astype('Int64')

# 3️⃣ '물건금액(만원)' 컬럼도 정수형(Int64) 유지
# - 원래 숫자를 그대로 유지하면서 결측값도 허용
nowon_propertyType_df['물건금액(만원)'] = nowon_propertyType_df['물건금액(만원)'].astype('Int64')

# 4️⃣ 보기용 컬럼 생성
# - 원본 '물건금액(만원)' 컬럼은 그대로 유지
# - 사람이 보기 편한 "억/만 원" 문자열을 '보기용' 컬럼에 저장
nowon_propertyType_df['보기용'] = nowon_propertyType_df['물건금액(만원)'].apply(format_korean_money)

# 5️⃣ 결과 확인
# - 전체 데이터프레임 출력
nowon_propertyType_df

,자치구명,법정동명,건물명,물건금액(만원),건물면적(㎡),건축년도,건물용도,보기용
85,노원구,공릉동,스위트휴,26300,29.53,2024,연립다세대,"2억 6,300만 원"
92,노원구,공릉동,스위트휴,44600,58.75,2024,연립다세대,"4억 4,600만 원"
93,노원구,공릉동,스위트휴,44650,57.81,2024,연립다세대,"4억 4,650만 원"
94,노원구,공릉동,스위트휴,40450,54.14,2024,연립다세대,4억 450만 원
95,노원구,공릉동,스위트휴,40450,54.14,2024,연립다세대,4억 450만 원
...,...,...,...,...,...,...,...,...
235529,노원구,공릉동,NaN,85000,197.28,1991,단독다가구,"8억 5,000만 원"
235564,노원구,상계동,호연빌라,50000,58.98,1995,연립다세대,5억 원
235570,노원구,중계동,성원1,88000,59.75,1996,아파트,"8억 8,000만 원"
235580,노원구,상계동,대우팰리스,50000,28.10,2017,연립다세대,5억 원


In [74]:
nowon_propertyType_df_count = nowon_propertyType_df['보기용'].value_counts()
nowon_propertyType_df_count

보기용
5억 원            174
6억 원            157
5억 5,000만 원     122
4억 9,000만 원     118
4억 원            111
               ... 
1억 6,470만 원       1
1억 4,950만 원       1
13억 2,500만 원      1
5억 5,300만 원       1
9억 5,300만 원       1
Name: count, Length: 1079, dtype: int64

In [75]:
# --- 8억~10억원(단위: 만원) 범위 필터링 ---


# 1️⃣ '건축년도', '물건금액(만원)', '건물면적(㎡)' 컬럼을 숫자로 변환
# - errors='coerce' 옵션: 변환 불가 값은 NaN으로 처리
nowon_propertyType_df['건축년도'] = pd.to_numeric(nowon_propertyType_df['건축년도'], errors='coerce')
nowon_propertyType_df['물건금액(만원)'] = pd.to_numeric(nowon_propertyType_df['물건금액(만원)'], errors='coerce')
nowon_propertyType_df['건물면적(㎡)'] = pd.to_numeric(nowon_propertyType_df['건물면적(㎡)'], errors='coerce')

# 2️⃣ 건물면적 값을 소수 둘째 자리까지 반올림
# - 값 자체는 float로 유지, 계산·비교 가능
nowon_propertyType_df['건물면적(㎡)'] = nowon_propertyType_df['건물면적(㎡)'].round(2)

# 3️⃣ 판다스 전체 float 출력 형식 설정
# - 화면에 표시할 때 소수 둘째 자리까지 보여줌
pd.options.display.float_format = '{:.2f}'.format

# 4️⃣ 금액을 "억 / 만원" 형태 문자열로 변환하는 함수
def format_korean_money(price):
    if pd.isna(price):
        return None  # 값이 NaN이면 None 반환

    price = int(price)        # 만원 단위로 변환
    eok = price // 10000      # 억 단위 계산
    man = price % 10000       # 억 제외 나머지 만 단위 계산

    # 억과 만 모두 있을 때
    if eok > 0 and man > 0:
        return f"{eok}억 {man:,}만 원"
    # 억만 있을 때
    elif eok > 0:
        return f"{eok}억 원"
    # 억 없고 만만 있을 때
    else:
        return f"{man:,}만 원"

# 5️⃣ '보기용' 컬럼 생성
# - 원본 '물건금액(만원)' 숫자는 그대로 유지
# - 사람이 보기 편한 문자열은 '보기용' 컬럼에 저장
nowon_propertyType_df['보기용'] = nowon_propertyType_df['물건금액(만원)'].apply(format_korean_money)

# 6️⃣ 8억~10억원(단위: 만원) 범위 필터링
df_filtered = nowon_propertyType_df[
    (nowon_propertyType_df['물건금액(만원)'] >= 80000) &
    (nowon_propertyType_df['물건금액(만원)'] <= 100000)
]

# 7️⃣ 필터 결과 확인
df_filtered



,자치구명,법정동명,건물명,물건금액(만원),건물면적(㎡),건축년도,건물용도,보기용
326,노원구,월계동,풍림아이원,83000,84.30,2005,아파트,"8억 3,000만 원"
856,노원구,하계동,하계1청구,88000,84.60,1997,아파트,"8억 8,000만 원"
906,노원구,중계동,양지마을(대림)1,94500,84.90,1998,아파트,"9억 4,500만 원"
999,노원구,월계동,삼호3,80500,59.22,1986,아파트,8억 500만 원
1114,노원구,공릉동,풍림아파트A,80000,84.99,2001,아파트,8억 원
...,...,...,...,...,...,...,...,...
235033,노원구,하계동,건영,92500,74.58,1988,아파트,"9억 2,500만 원"
235516,노원구,공릉동,태릉 해링턴 플레이스,100000,59.98,0,아파트,10억 원
235529,노원구,공릉동,NaN,85000,197.28,1991,단독다가구,"8억 5,000만 원"
235570,노원구,중계동,성원1,88000,59.75,1996,아파트,"8억 8,000만 원"


In [76]:
nowon_filtered_df = df_filtered[df_filtered['법정동명'].str.contains('중계동', na=False)]
nowon_filtered_df

,자치구명,법정동명,건물명,물건금액(만원),건물면적(㎡),건축년도,건물용도,보기용
906,노원구,중계동,양지마을(대림)1,94500,84.90,1998,아파트,"9억 4,500만 원"
2297,노원구,중계동,주공5,97500,76.51,1992,아파트,"9억 7,500만 원"
2857,노원구,중계동,중앙하이츠아쿠아,92500,84.47,2008,아파트,"9억 2,500만 원"
3728,노원구,중계동,중앙하이츠아쿠아,89000,84.47,2008,아파트,"8억 9,000만 원"
4900,노원구,중계동,신안,94300,84.93,1999,아파트,"9억 4,300만 원"
...,...,...,...,...,...,...,...,...
221294,노원구,중계동,신안,81000,59.97,1999,아파트,"8억 1,000만 원"
221295,노원구,중계동,신안,81000,59.97,1999,아파트,"8억 1,000만 원"
224193,노원구,중계동,양지마을(대림)2,99000,84.88,1999,아파트,"9억 9,000만 원"
234598,노원구,중계동,삼성,94000,84.75,1999,아파트,"9억 4,000만 원"


In [77]:
# 1️⃣ 'nowon_filtered_df'에서 건축년도 2000~2025년 범위만 필터링
# - 조건: 건축년도 >= 2000 AND 건축년도 <= 2025
# - & 연산자를 쓸 때 각 조건은 괄호로 묶어야 함
df_filtered_year = nowon_filtered_df[
    (nowon_filtered_df['건축년도'] >= 2000) &
    (nowon_filtered_df['건축년도'] <= 2025)
].copy()  # copy() 사용: 원본 df를 안전하게 유지

# df_filtered_year의 인덱스를 1부터 시작하도록 재설정
df_filtered_year.index = range(1, len(df_filtered_year) + 1)


# 2️⃣ 결과 확인
df_filtered_year

,자치구명,법정동명,건물명,물건금액(만원),건물면적(㎡),건축년도,건물용도,보기용
1,노원구,중계동,중앙하이츠아쿠아,92500,84.47,2008,아파트,"9억 2,500만 원"
2,노원구,중계동,중앙하이츠아쿠아,89000,84.47,2008,아파트,"8억 9,000만 원"
3,노원구,중계동,한화꿈에그린,83000,84.90,2005,아파트,"8억 3,000만 원"
4,노원구,중계동,한화꿈에그린,83000,84.90,2005,아파트,"8억 3,000만 원"
5,노원구,중계동,경남아너스빌,80000,105.60,2002,아파트,8억 원
6,노원구,중계동,한화꿈에그린,84000,84.90,2005,아파트,"8억 4,000만 원"
7,노원구,중계동,현대(6차),99500,122.68,2018,아파트,"9억 9,500만 원"
8,노원구,중계동,한화꿈에그린,88500,84.90,2005,아파트,"8억 8,500만 원"
9,노원구,중계동,한화꿈에그린,89500,84.90,2005,아파트,"8억 9,500만 원"
10,노원구,중계동,동도센트리움(601),82000,84.95,2005,아파트,"8억 2,000만 원"
